In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sympy.physics.units as u
import sympy as sp
import numpy as np
import logging as log
import functools
import matplotlib.pyplot as plt
import seaborn
import pandas as pd
pd.set_option('display.max_columns', None)
%matplotlib inline
from IPython.display import display, HTML

import sys, os, importlib
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import miptlabs as ml
importlib.reload(ml)

importlib.reload(log)
log.basicConfig(level=log.INFO, format='LOG %(message)s', stream=sys.stdout)

sp.Basic.__str__=lambda expr, **settings:ml.PQStrPrinter(settings).doprint(expr)

sp.__version__

'1.1.2.dev'

In [3]:
data = pd.read_csv('5.2.csv')
display(data)

,No,"F_exp, mm","F_exp^inv, mm","F_true, mm"
0,1,97,99,100
1,2,246,248,250
2,3,148,150,150
3,4,0,0,130


In [4]:
# Погрешность измерения длины одинакова: по 5 мм с каждой стороны
dist_sigma = np.sqrt(2)*5*u.mm

df2 = pd.DataFrame()
df2['F_exp, mm'] = ml.get_column(data, 'F_exp, mm', lambda row: ml.PQ(row['F_exp, mm']*u.mm, sigma=dist_sigma))
df2['F_exp^inv, mm'] = ml.get_column(data, 'F_exp^inv, mm', lambda row: ml.PQ(row['F_exp^inv, mm']*u.mm, sigma=dist_sigma))
df2['F_true, mm'] = ml.get_column(data, 'F_true, mm', lambda row: ml.PQ(row['F_true, mm']*u.mm, is_const=True))

# Расчёт для рассеивающей линзы
a_0 = ml.PQ(295*u.mm, sigma=dist_sigma)
l = ml.PQ(163*u.mm, sigma=dist_sigma)
l_inv = ml.PQ(165*u.mm, sigma=dist_sigma)
df2['F_exp, mm'][3] = a_0-l
df2['F_exp^inv, mm'][3] = a_0-l_inv

display(df2)

,"F_exp, mm","F_exp^inv, mm","F_true, mm"
0,97±7 mm (7.3%),99±7 mm (7.1%),100.000000 mm
1,246±7 mm (2.9%),248±7 mm (2.9%),250.000000 mm
2,148±7 mm (4.8%),150±7 mm (4.7%),150.000000 mm
3,132±10 mm (7.6%),130±10 mm (7.7%),130.000000 mm


In [10]:
# Труба Кеплера
f = ml.PQ(154*u.mm, sigma=dist_sigma) # коллиматор
f_12 = ml.PQ(358*u.mm, sigma=dist_sigma)
f_objective = df2['F_exp, mm'][1]
f_ocular = df2['F_exp, mm'][0]
f_12_true = f_objective+f_ocular
#print(f_12, f_12_true)
gamma_theor = f_objective/f_ocular
print(gamma_theor)
gamma_exper = 1

2.54±0.20  (7.8%)


In [11]:
# Труба Галилея
f = ml.PQ(154*u.mm, sigma=dist_sigma) # коллиматор
f_12 = ml.PQ(124*u.mm, sigma=dist_sigma)
f_objective = df2['F_exp, mm'][1]
f_ocular = df2['F_exp, mm'][3]
f_12_true = f_objective-f_ocular
print(f_12, f_12_true)
gamma_theor = f_objective/f_ocular
print(gamma_theor)

124±7 mm (5.7%) 114±12 mm (11%)
1.86±0.15  (8.1%)


In [12]:
# Проекционный микроскоп
d1 = ml.PQ(123*u.mm, sigma=dist_sigma)
f1d2 = ml.PQ(976*u.mm, sigma=dist_sigma)
f1 = ml.PQ(680*u.mm, sigma=dist_sigma)
d2 = f1d2-f1
print(d2)
f2 = ml.PQ(402*u.mm, sigma=dist_sigma)
gamma = f1/d1*f2/d2
print(gamma)

x_d = ml.PQ(0.5*u.mm, is_const=True)
x_f = ml.PQ(20*u.mm, sigma=1*np.sqrt(2)*u.mm)/5
gamma_exper = x_f/x_d
print(gamma_exper)

296±10 mm (3.4%)
7.5±0.5  (7.0%)
8.0±0.6  (7.1%)


In [8]:
F2 = ml.PQ(100*u.mm, sigma=5*u.mm)
a = ml.PQ(135*u.mm, sigma=5*u.mm)
bplusc = ml.PQ(590*u.mm, sigma=5*u.mm)
d = ml.PQ(415*u.mm, sigma=5*u.mm)

#a = 135
#F2 = 100
log.getLogger().setLevel(log.INFO)
s = np.sqrt(( (a*(a-F2)-F2*a)/(a-F2)**2 )**2*5**2+ ( (F2*(a-F2)+a*F2)/(a-F2)**2)**2*5**2)
print('s', s.val)

b = (F2*a)/(a-F2)
print('b', b.sigma)
b = (a*F2)/(a-F2)
print('b', b.sigma)
b = ml.eval(u.mm, lambda a, F2: (F2*a)/(a-F2), a, F2)
print('b', b.sigma)
c = bplusc - b
print('c', c)
gamma = b/a*d/c
print('gamma', gamma)

s 78.0862971605661
b 81.5382745243698*mm
b 81.5382745243698*mm
b 84.8499299974653*mm
c 200±80 mm (42%)
gamma 5.8±2.7  (47%)


In [9]:
F1 = ml.PQ(100*u.mm, sigma=5*u.mm)
F2 = ml.PQ(150*u.mm, sigma=5*u.mm)
delta = ml.PQ(465*u.mm, sigma=5*u.mm)
l = ml.PQ(135*u.mm, sigma=5*u.mm)

t = 

SyntaxError: invalid syntax (<ipython-input-9-95d0326029c8>, line 6)